In [1]:
import pandas as pd

from classification.OneRClassifier import OneRClassifier
from classification.NaiveBayesClassifier import NaiveBayesClassifier

In [2]:
classifiers = [OneRClassifier()]  # TODO

#### Check classifiers (Sklearn)

In [3]:
import warnings
from sklearn.utils.estimator_checks import estimator_checks_generator

for clf in classifiers:
    for (estimator, check) in estimator_checks_generator(clf):
        try:
            check(estimator)
        except KeyError as e:
            if "encountered an unknown value" in str(e):
                continue
            warnings.warn(str(e))

#### data_4x

In [4]:
df = pd.read_csv('data/data_4x.csv')
df.head()

,x0,x1,x2,x3,y
0,0,0,1,0,0
1,1,0,0,1,1
2,2,0,1,0,1
3,0,1,0,0,1
4,0,1,1,0,1


In [5]:
X, y = df.drop(columns='y'), df['y']
X.shape, y.shape

((10, 4), (10,))

In [6]:
y.replace(0, 'class_0', inplace=True)
y.replace(1, 'class_1', inplace=True)
y = list(y)

##### OneRClassifier

In [7]:
model = OneRClassifier()
model.fit(X, y);

In [8]:
model.predict(pd.DataFrame({'x0': [2], 'x1': [1], 'x2': [1], 'x3': [1]}))

array(['class_0'], dtype='<U7')

##### NaiveBayesClassifier

In [9]:
model = NaiveBayesClassifier()
model.fit(X, y);

[y   
0  0    3
   1    1
1  2    3
   0    2
   1    1
Name: attr, dtype: int64, y  0
0  0    2
   1    2
1  0    3
   1    3
Name: attr, dtype: int64, y   
0  1    3
   0    1
1  0    3
   1    3
Name: attr, dtype: int64, y   
0  1    3
   0    1
1  0    5
   1    1
Name: attr, dtype: int64]
[0.3333333333333333, 0.5, 0.5, 0.5]


In [10]:
model.predict(pd.DataFrame({'x0': [2], 'x1': [1], 'x2': [1], 'x3': [1]}))

array(Ellipsis, dtype=object)

##### DecisionTreeClassifier

In [11]:
# TODO

##### KNeighborsClassifier

In [12]:
# TODO

#### data_3x

In [13]:
df = pd.read_csv('data/data_3x.csv')
df.head()

,dataset,x0,x1,x2,y
0,1,0,1,2,1
1,1,1,0,1,1
2,1,0,1,1,0
3,1,0,0,1,1
4,1,0,0,2,1


In [14]:
res_df = pd.DataFrame(
    index=[str(clf) for clf in classifiers],
    columns=df['dataset'].unique()
)

X_pred = pd.DataFrame({'x0': [1], 'x1': [1], 'x2': [1]})
for clf in classifiers:
    for df_i, df_v in df.groupby('dataset'):
        df_v = df_v.drop(columns=['dataset'])
        X, y = df_v.drop(columns='y'), df_v['y']
        clf.fit(X, y)
        y_pred = clf.predict(X_pred)[0]
        res_df.loc[str(clf), df_i] = y_pred

res_df

,1,2,3,4
OneRClassifier(),1,1,1,1
